# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - API Gateways

**Subtopic** - Implementing API Gateway Rate Limiting

**Use Case** - Develop an anomaly detection module using scikit-learn to identify and alert on unusual spikes in API request rates within an Istio service mesh.

**Programming Language** - Python

**Target Model** - o1

# Setup

```requirements.txt
```


# Prompt

TechAI Corp operates an AI platform serving three subscriber tiers (Free, Professional, Enterprise) with different usage limits and priorities. Different AI operations consume varying computational resources based on input content length. The system must enforce rate limits across multiple time windows (per-second, per-minute, per-day) while providing priority queuing during high traffic. You must implement a decorator-based rate limiting system that handles tokenization, multi-tier limits, and intelligent queuing to protect API endpoints while ensuring fair resource allocation.

**Input Format**
* Methods: list of method names to call ["AIRateLimiter", "create_limiter", "generate_text"]
* Parameters: list of parameter lists for each method call
* Constructor parameters: tier_configs, time_windows, operation_costs, limiter_type

**Output Format**
* Results: list of return values for each method call
* Constructor/void methods return null
* Successful calls: original response + rate limiting metadata
* Queued calls: queue position, wait time, priority level
* Rejected calls: error response with retry timing

**Examples**

```python
# Example 1: Request-based rate limiting
Input: ["AIRateLimiter", "create_limiter", "generate_text", "generate_text", "generate_text"]
       [[{"pro": {"limits": [2, 10], "priority": 2}}, [1, 60], {}, "request"], [], ["user1", "pro", "text_gen", "hello"], ["user1", "pro", "text_gen", "world"], ["user1", "pro", "text_gen", "test"]]
Output: [null, decorator_function, {"result": "hello", "rate_limit_info": {"requests_consumed": 1, "remaining_requests": [1, 9]}}, {"result": "world", "rate_limit_info": {"requests_consumed": 1, "remaining_requests": [0, 8]}}, {"status": "queued", "queue_position": 1, "estimated_wait_time": 1}]

# Example 2: Token-based rate limiting  
Input: ["AIRateLimiter", "create_limiter", "generate_text"]
       [[{"pro": {"token_limits": [5, 50], "priority": 2}}, [1, 60], {"text_gen": 2}, "token"], [], ["user1", "pro", "text_gen", "one two three"]]
Output: [null, decorator_function, {"status": "queued", "queue_position": 1, "estimated_wait_time": 1}]

# Example 3: Successful token consumption
Input: ["AIRateLimiter", "create_limiter", "get_token_count", "generate_text"]
       [[{"pro": {"token_limits": [5, 50], "priority": 2}}, [1, 60], {"text_gen": 2}, "token"], [], ["hello"], ["user1", "pro", "text_gen", "hello"]]
Output: [null, decorator_function, 1, {"result": "hello", "rate_limit_info": {"tokens_consumed": 2, "remaining_tokens": [3, 48]}}]
```



# Requirements

**Explicit and Implicit Points**
* Implement factory pattern creating two rate limiter types: "token" and "request" based
* Token type: calculates consumption as content_word_count × operation_cost
* Request type: each API call consumes 1 from limits regardless of content size
* Support three time windows simultaneously with independent limits
* Implement priority queuing when limits exceeded (enterprise > pro > free)
* Return appropriate metadata based on limiter type (tokens_consumed vs requests_consumed)

**Solution Expectations**
* Create AIRateLimiter class as factory for generating decorator functions
* Implement get_token_count method for content tokenization by splitting on whitespace
* Design extensible architecture allowing dynamic configuration changes
* Store per-user state across multiple time windows using dictionaries
* Handle queue management with priority levels and wait time estimation
* Preserve original function behavior while adding rate limit metadata

**Function Signatures**
```python
class AIRateLimiter:
    def __init__(self, tier_configs: dict, time_windows: list, operation_costs: dict, limiter_type: str):
        # Initialize factory with configurations and limiter type ("token" or "request")
        pass
    
    def get_token_count(self, content: str) -> int:
        # Tokenization method - split content and return token count
        pass
    
    def create_limiter(self):
        # Factory method that returns appropriate decorator based on limiter_type
        pass
```

**Edge Case Behavior**
* New users start with full token allocation for their tier
* Handle time window boundary crossings and limit resets accurately
* Empty content defaults to 1 token minimum consumption
* Process queued requests by priority then FIFO order

**Constraints**
* Use only Python standard library modules
* Store all state in memory using dictionaries
* Support up to 1000 users with 10000 total requests
* Content limited to 1000 words maximum
* All configuration values must be positive integers
* Decorated functions must have (user_id, tier, operation, content) as first parameters

In [ ]:
# code

"""AI Rate Limiter Implementation for Task 69308."""

import time
import heapq
from functools import wraps
from typing import Dict, List, Tuple, Any, Callable


class AIRateLimiter:
    """Factory class for creating AI platform rate limiters."""

    def __init__(
        self,
        tier_configs: Dict[str, Dict],
        time_windows: List[int],
        operation_costs: Dict[str, int],
        limiter_type: str,
    ):
        """Initialize the rate limiter factory."""
        self.tier_configs = tier_configs
        self.time_windows = time_windows
        self.operation_costs = operation_costs
        self.limiter_type = limiter_type
        self.user_state: Dict[str, Dict[int, List[Tuple[float, int]]]] = {}
        self.request_queue: List[Tuple[int, float, str, Dict]] = []
        self._priority_map = {
            tier: config["priority"] for tier, config in tier_configs.items()
        }
        self.user_tiers: Dict[str, str] = {}

    def _validate_inputs(
        self, user_id: str, tier: str, operation: str, content: str
    ) -> None:
        """Validate all inputs before processing."""
        if not isinstance(user_id, str) or not user_id.strip():
            raise ValueError("user_id must be a non-empty string")

        if not isinstance(tier, str) or tier not in self.tier_configs:
            raise ValueError(
                f"Unknown tier: {tier}. Available tiers: "
                f"{list(self.tier_configs.keys())}"
            )

        if not isinstance(operation, str) or not operation.strip():
            raise ValueError("operation must be a non-empty string")

        if not isinstance(content, str):
            raise ValueError("content must be a string")

        tier_config = self.tier_configs[tier]
        required_key = "token_limits" if self.limiter_type == "token" else "limits"
        if required_key not in tier_config:
            raise ValueError(f"Tier {tier} missing {required_key} configuration")

        limits = tier_config[required_key]
        if len(limits) != len(self.time_windows):
            raise ValueError(
                f"Tier {tier} has {len(limits)} limits but "
                f"{len(self.time_windows)} time windows"
            )

    def get_token_count(self, content: str) -> int:
        """Calculate token count from content."""
        if not content or not content.strip():
            return 1
        return len(content.split())

    def create_limiter(self) -> Callable:
        """Create appropriate decorator based on limiter_type."""
        if self.limiter_type == "token":
            return self._create_token_limiter()
        elif self.limiter_type == "request":
            return self._create_request_limiter()
        else:
            raise ValueError(f"Unknown limiter type: {self.limiter_type}")

    def _create_token_limiter(self) -> Callable:
        """Create token-based rate limiter decorator."""

        @wraps(self._create_token_limiter)
        def token_limiter(func: Callable) -> Callable:
            @wraps(func)
            def wrapper(
                user_id: str, tier: str, operation: str, content: str
            ) -> Dict[str, Any]:
                self._validate_inputs(user_id, tier, operation, content)
                current_time = time.time()
                self.user_tiers[user_id] = tier
                token_count = self.get_token_count(content)
                consumption = token_count * self.operation_costs.get(operation, 1)

                if self._check_rate_limits(user_id, tier, consumption, current_time):
                    self._record_consumption(user_id, consumption, current_time)
                    result = func(user_id, tier, operation, content)
                    remaining_tokens = self._calculate_remaining_capacity(
                        user_id, tier, current_time
                    )
                    return {
                        "result": result,
                        "rate_limit_info": {
                            "tokens_consumed": consumption,
                            "remaining_tokens": remaining_tokens,
                        },
                    }
                else:
                    return self._add_to_queue(
                        user_id,
                        tier,
                        {
                            "operation": operation,
                            "content": content,
                            "consumption": consumption,
                        },
                    )

            return wrapper

        return token_limiter

    def _create_request_limiter(self) -> Callable:
        """Create request-based rate limiter decorator."""

        @wraps(self._create_request_limiter)
        def request_limiter(func: Callable) -> Callable:
            @wraps(func)
            def wrapper(
                user_id: str, tier: str, operation: str, content: str
            ) -> Dict[str, Any]:
                self._validate_inputs(user_id, tier, operation, content)
                current_time = time.time()
                self.user_tiers[user_id] = tier
                consumption = 1

                if self._check_rate_limits(user_id, tier, consumption, current_time):
                    self._record_consumption(user_id, consumption, current_time)
                    result = func(user_id, tier, operation, content)
                    remaining_requests = self._calculate_remaining_capacity(
                        user_id, tier, current_time
                    )
                    return {
                        "result": result,
                        "rate_limit_info": {
                            "requests_consumed": consumption,
                            "remaining_requests": remaining_requests,
                        },
                    }
                else:
                    return self._add_to_queue(
                        user_id,
                        tier,
                        {
                            "operation": operation,
                            "content": content,
                            "consumption": consumption,
                        },
                    )

            return wrapper

        return request_limiter

    def _check_rate_limits(
        self, user_id: str, tier: str, consumption: int, current_time: float
    ) -> bool:
        """Check if request is within rate limits for all time windows."""
        self._cleanup_expired_entries(user_id, current_time)
        tier_config = self.tier_configs.get(tier, {})
        limits = (
            tier_config.get("limits", [])
            if self.limiter_type == "request"
            else tier_config.get("token_limits", [])
        )

        for window_idx, limit in enumerate(limits):
            current_usage = self._get_window_usage(user_id, window_idx)
            if current_usage + consumption > limit:
                return False

        return True

    def _cleanup_expired_entries(self, user_id: str, current_time: float) -> None:
        """Remove expired entries from user state."""
        if user_id not in self.user_state:
            return

        for window_idx, window_seconds in enumerate(self.time_windows):
            if window_idx in self.user_state[user_id]:
                cutoff_time = current_time - window_seconds
                self.user_state[user_id][window_idx] = [
                    (timestamp, consumption)
                    for timestamp, consumption in self.user_state[user_id][window_idx]
                    if timestamp > cutoff_time
                ]

    def _get_window_usage(self, user_id: str, window_idx: int) -> int:
        """Get current usage for a specific time window."""
        if user_id not in self.user_state or window_idx not in self.user_state[user_id]:
            return 0
        return sum(
            consumption for _, consumption in self.user_state[user_id][window_idx]
        )

    def _record_consumption(
        self, user_id: str, consumption: int, timestamp: float
    ) -> None:
        """Record consumption across all time windows."""
        if user_id not in self.user_state:
            self.user_state[user_id] = {}

        for window_idx in range(len(self.time_windows)):
            if window_idx not in self.user_state[user_id]:
                self.user_state[user_id][window_idx] = []
            self.user_state[user_id][window_idx].append((timestamp, consumption))

    def _calculate_remaining_capacity(
        self, user_id: str, tier: str, current_time: float
    ) -> List[int]:
        """Calculate remaining capacity for each time window."""
        self._cleanup_expired_entries(user_id, current_time)
        tier_config = self.tier_configs.get(tier, {})
        limits = (
            tier_config.get("limits", [])
            if self.limiter_type == "request"
            else tier_config.get("token_limits", [])
        )

        remaining = []
        for window_idx, limit in enumerate(limits):
            current_usage = self._get_window_usage(user_id, window_idx)
            remaining.append(max(0, limit - current_usage))

        return remaining

    def _add_to_queue(
        self, user_id: str, tier: str, request_data: Dict
    ) -> Dict[str, Any]:
        """Add request to priority queue."""
        current_time = time.time()
        priority = self._priority_map.get(tier, 999)

        queue_position = (
            sum(
                1
                for p, t, _, _ in self.request_queue
                if p < priority or (p == priority and t < current_time)
            )
            + 1
        )

        heapq.heappush(
            self.request_queue, (priority, current_time, user_id, request_data)
        )

        estimated_wait_time = min(self.time_windows)

        return {
            "status": "queued",
            "queue_position": queue_position,
            "estimated_wait_time": estimated_wait_time,
        }

    def process_queue(self) -> List[Dict]:
        """Process queued requests that can now be executed."""
        processed = []
        current_time = time.time()

        while self.request_queue:
            priority, timestamp, user_id, request_data = self.request_queue[0]
            tier = self.user_tiers.get(user_id, "unknown")
            if tier == "unknown":
                heapq.heappop(self.request_queue)
                continue

            if self._check_rate_limits(
                user_id, tier, request_data["consumption"], current_time
            ):
                heapq.heappop(self.request_queue)
                self._record_consumption(
                    user_id, request_data["consumption"], current_time
                )
                processed.append(
                    {
                        "user_id": user_id,
                        "tier": tier,
                        "operation": request_data["operation"],
                        "content": request_data["content"],
                        "processed": True,
                    }
                )
            else:
                break

        return processed

## Optional code to run test cases, and check main file for issue

def generate_text(user_id: str, tier: str, operation: str, content: str) -> str:
    """Mock function that generates text."""
    return content


def run_examples():
    """Run the examples from the problem statement."""
    results = []

    # Example 1: Request-based rate limiting
    methods = [
        "AIRateLimiter",
        "create_limiter",
        "generate_text",
        "generate_text",
        "generate_text",
    ]
    parameters = [
        [{"pro": {"limits": [2, 10], "priority": 2}}, [1, 60], {}, "request"],
        [],
        ["user1", "pro", "text_gen", "hello"],
        ["user1", "pro", "text_gen", "world"],
        ["user1", "pro", "text_gen", "test"],
    ]

    # Execute methods
    limiter = None
    decorator_func = None

    for i, (method, params) in enumerate(zip(methods, parameters)):
        if method == "AIRateLimiter":
            limiter = AIRateLimiter(*params)
            results.append(None)
        elif method == "create_limiter":
            decorator_func = limiter.create_limiter()
            results.append("decorator_function")
        elif method == "generate_text":
            if decorator_func:
                decorated_func = decorator_func(generate_text)
                result = decorated_func(*params)
                results.append(result)

    return results


def run_all_examples():
    """Run all examples from the problem statement."""
    print("=== Testing All Examples ===\n")

    # Example 1: Request-based rate limiting
    print("Example 1: Request-based rate limiting")
    methods1 = [
        "AIRateLimiter",
        "create_limiter",
        "generate_text",
        "generate_text",
        "generate_text",
    ]
    parameters1 = [
        [{"pro": {"limits": [2, 10], "priority": 2}}, [1, 60], {}, "request"],
        [],
        ["user1", "pro", "text_gen", "hello"],
        ["user1", "pro", "text_gen", "world"],
        ["user1", "pro", "text_gen", "test"],
    ]

    results1 = []
    limiter1 = None
    decorator_func1 = None

    for i, (method, params) in enumerate(zip(methods1, parameters1)):
        if method == "AIRateLimiter":
            limiter1 = AIRateLimiter(*params)
            results1.append(None)
        elif method == "create_limiter":
            decorator_func1 = limiter1.create_limiter()
            results1.append("decorator_function")
        elif method == "generate_text":
            if decorator_func1:
                decorated_func1 = decorator_func1(generate_text)
                result = decorated_func1(*params)
                results1.append(result)

    for i, result in enumerate(results1):
        print(f"  {i+1}: {result}")

    # Example 2: Token-based rate limiting
    print("\nExample 2: Token-based rate limiting")
    methods2 = ["AIRateLimiter", "create_limiter", "generate_text"]
    parameters2 = [
        [
            {"pro": {"token_limits": [5, 50], "priority": 2}},
            [1, 60],
            {"text_gen": 2},
            "token",
        ],
        [],
        ["user1", "pro", "text_gen", "one two three"],
    ]

    results2 = []
    limiter2 = None
    decorator_func2 = None

    for i, (method, params) in enumerate(zip(methods2, parameters2)):
        if method == "AIRateLimiter":
            limiter2 = AIRateLimiter(*params)
            results2.append(None)
        elif method == "create_limiter":
            decorator_func2 = limiter2.create_limiter()
            results2.append("decorator_function")
        elif method == "generate_text":
            if decorator_func2:
                decorated_func2 = decorator_func2(generate_text)
                result = decorated_func2(*params)
                results2.append(result)

    for i, result in enumerate(results2):
        print(f"  {i+1}: {result}")

    # Example 3: Successful token consumption
    print("\nExample 3: Successful token consumption")
    methods3 = ["AIRateLimiter", "create_limiter", "get_token_count", "generate_text"]
    parameters3 = [
        [
            {"pro": {"token_limits": [5, 50], "priority": 2}},
            [1, 60],
            {"text_gen": 2},
            "token",
        ],
        [],
        ["hello"],
        ["user1", "pro", "text_gen", "hello"],
    ]

    results3 = []
    limiter3 = None
    decorator_func3 = None

    for i, (method, params) in enumerate(zip(methods3, parameters3)):
        if method == "AIRateLimiter":
            limiter3 = AIRateLimiter(*params)
            results3.append(None)
        elif method == "create_limiter":
            decorator_func3 = limiter3.create_limiter()
            results3.append("decorator_function")
        elif method == "get_token_count":
            token_count = limiter3.get_token_count(*params)
            results3.append(token_count)
        elif method == "generate_text":
            if decorator_func3:
                decorated_func3 = decorator_func3(generate_text)
                result = decorated_func3(*params)
                results3.append(result)

    for i, result in enumerate(results3):
        print(f"  {i+1}: {result}")


if __name__ == "__main__":
    example_results = run_examples()
    print("Example Results:")
    for i, result in enumerate(example_results):
        print(f"  {i+1}: {result}")

    print("\n" + "=" * 50 + "\n")

    # Test all examples
    run_all_examples()


In [1]:
# tests

"""Comprehensive tests for AIRateLimiter in a single TestCase class."""
import unittest
from unittest.mock import patch
from main import AIRateLimiter


class _FakeClock:
    def __init__(self, start: int = 0) -> None:
        self._now = start

    def time(self) -> int:
        return self._now

    def tick(self, seconds: int = 1) -> None:
        self._now += seconds


class TestAIRateLimiter(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        cls.clock = _FakeClock()
        cls._patcher = patch("time.time", cls.clock.time)
        cls._patcher.start()

    @classmethod
    def tearDownClass(cls):
        cls._patcher.stop()

    # Helper fabricators
    def _request_limiter(self):
        cfg = {
            "free": {"limits": [2, 5, 10], "priority": 1},
            "enterprise": {"limits": [2, 5, 10], "priority": 3},
        }
        win = [1, 60, 86400]
        return main.AIRateLimiter(cfg, win, {}, "request").create_limiter()

    def _token_limiter(self):
        cfg = {"pro": {"token_limits": [5, 50, 100], "priority": 2}}
        win = [1, 60, 86400]
        costs = {"text_gen": 2}
        rl = main.AIRateLimiter(cfg, win, costs, "token")
        return rl.create_limiter(), rl

    # Request-based tests
    def test_request_success_counts(self):
        deco = self._request_limiter()

        @deco
        def api(u, t, o, c):
            return c

        r = api("u1", "free", "op", "x")
        self.assertEqual(r["rate_limit_info"]["requests_consumed"], 1)
        self.assertEqual(r["rate_limit_info"]["remaining_requests"][0], 1)

    def test_request_window_exhaustion_queue(self):
        deco = self._request_limiter()

        @deco
        def api(u, t, o, c):
            return c

        api("u2", "free", "op", "a")
        api("u2", "free", "op", "b")
        q = api("u2", "free", "op", "c")
        self.assertEqual(q["status"], "queued")

    def test_request_reset_after_window(self):
        deco = self._request_limiter()

        @deco
        def api(u, t, o, c):
            return c

        api("u3", "free", "op", "a")
        api("u3", "free", "op", "b")
        self.clock.tick(1)
        r = api("u3", "free", "op", "c")
        self.assertIn("rate_limit_info", r)

    # Token-based tests
    def test_token_success_counts(self):
        deco, _ = self._token_limiter()

        @deco
        def api(u, t, o, c):
            return c

        r = api("u4", "pro", "text_gen", "one two")
        self.assertEqual(r["tokens_consumed"], 4)
        self.assertEqual(r["remaining_tokens"][0], 1)

    def test_token_empty_content_minimum(self):
        deco, rl = self._token_limiter()

        @deco
        def api(u, t, o, c):
            return c

        self.assertEqual(rl.get_token_count(""), 1)
        r = api("u5", "pro", "text_gen", "")
        self.assertEqual(r["tokens_consumed"], 2)  # 1 token × cost 2

    def test_token_queue_on_exceed(self):
        deco, _ = self._token_limiter()

        @deco
        def api(u, t, o, c):
            return c

        api("u6", "pro", "text_gen", "one two")
        q = api("u6", "pro", "text_gen", "one two three")
        self.assertEqual(q["status"], "queued")

    # Priority and FIFO tests
    def test_priority_enterprise_ahead_of_free(self):
        deco = self._request_limiter()

        @deco
        def api(u, t, o, c):
            return c

        api("f1", "free", "op", "x")
        api("f2", "free", "op", "y")
        q_free = api("f3", "free", "op", "z")
        q_ent = api("e1", "enterprise", "op", "z")
        self.assertLess(q_ent["queue_position"], q_free["queue_position"])

    def test_fifo_within_same_priority(self):
        deco = self._request_limiter()

        @deco
        def api(u, t, o, c):
            return c

        api("f4", "free", "op", "x")
        api("f5", "free", "op", "y")
        q1 = api("f6", "free", "op", "z")
        q2 = api("f7", "free", "op", "z")
        self.assertLess(q1["queue_position"], q2["queue_position"])

    # Helper method accuracy
    def test_helper_token_count_accuracy(self):
        _, rl = self._token_limiter()
        self.assertEqual(rl.get_token_count("one two three"), 3)

    def test_helper_token_count_empty(self):
        _, rl = self._token_limiter()
        self.assertEqual(rl.get_token_count(""), 1)

    # New user state
    def test_new_user_full_allocation(self):
        deco = self._request_limiter()

        @deco
        def api(u, t, o, c):
            return c

        r = api("newbie", "free", "op", "hi")
        self.assertEqual(r["rate_limit_info"]["remaining_requests"][0], 1)

    # Long window reset
    def test_daily_window_reset(self):
        deco = self._request_limiter()

        @deco
        def api(u, t, o, c):
            return c

        api("u7", "free", "op", "a")
        self.clock.tick(86400)
        r = api("u7", "free", "op", "b")
        self.assertEqual(r["rate_limit_info"]["remaining_requests"][2], 9)

    # Multi-user isolation
    def test_multi_user_isolated_state(self):
        deco = self._request_limiter()

        @deco
        def api(u, t, o, c):
            return c

        r1 = api("userA", "free", "op", "x")
        r2 = api("userB", "free", "op", "x")
        self.assertEqual(r1["rate_limit_info"]["remaining_requests"][0], 1)
        self.assertEqual(r2["rate_limit_info"]["remaining_requests"][0], 1)


if __name__ == "__main__":
    unittest.main()


ModuleNotFoundError: No module named 'main'

# Model Breaking Proof

#### Model Breaking Task URL: <https://rlhf-v3.turing.com/prompt/b8ad475f-17f2-42bb-9adf-c79be63d9a89?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F66273%2Fview>

#### Model code:

```python
# code generated by the model

import time
from collections import deque, defaultdict

class AIRateLimiter:
    def __init__(self, tier_configs: dict, time_windows: list, operation_costs: dict, limiter_type: str):
        """
        tier_configs: ex) {
            'free': {'limits': [5, 100, 1000], 'priority': 1},
            'pro': {'limits': [20, 300, 3000], 'priority': 2},
            'enterprise': {'limits': [100, 2000, 10000], 'priority': 3},
            ...
        }
        or for token-based:
            'pro': {'token_limits': [5, 50], 'priority': 2}, ...
        
        time_windows: list of time window sizes in seconds (e.g., [1, 60, 86400] for second, minute, day)
        operation_costs: dict mapping operation -> token cost (for token-based mode)
        limiter_type: "request" or "token"
        """
        # <Issue>: Missing validation of positive integer constraints for configuration values
        self.tier_configs = tier_configs
        self.time_windows = time_windows  # e.g. [1, 60, 86400]
        self.operation_costs = operation_costs  # e.g. {"text_gen": 2}
        self.limiter_type = limiter_type  # "token" or "request"

        # usage stores rolling usage for each user & tier & window index
        # usage[(user_id, tier, i)] = {"count": <count_in_current_window>, "start_time": <epoch>}
        self.usage = {}

        # A global queue for requests that exceed rate limits.
        # In a real system, you might process this queue asynchronously.
        # For simplicity, we store them, and return "queued" responses.
        self.request_queue = deque()

        # This will track a global incremental ID for queued requests.
        # In a real system, you might want more robust queue management.
        # <Issue>: Global counter doesn't provide correct per-user queue positions
        self.queue_counter = 0

    def get_token_count(self, content: str) -> int:
        """
        Tokenization method - split content and return token count.
        Content with zero tokens defaults to 1 token consumption.
        """
        tokens = content.split()
        count = len(tokens)
        return count if count > 0 else 1

    def _check_and_consume_tokens(self, user_id: str, tier: str, operation: str, content: str):
        """
        Internal helper: checks token usage against the user's tier token limits.
        Returns (proceed: bool, data: dict).
           If proceed == True, data contains rate limit metadata.
           If proceed == False, data contains queue or reject info.
        """
        now = time.time()
        tier_conf = self.tier_configs.get(tier)
        if not tier_conf or "token_limits" not in tier_conf:
            return (False, {"status": "error", "message": f"No token limits for tier '{tier}'"})

        token_limits = tier_conf["token_limits"]  # e.g. [5, 50, 500]
        priority = tier_conf.get("priority", 1)

        # Calculate how many tokens are required for this request
        token_count = self.get_token_count(content)
        operation_cost = self.operation_costs.get(operation, 1)
        tokens_needed = token_count * operation_cost

        # Check usage in each time window
        for i, window_size in enumerate(self.time_windows):
            limit = token_limits[i]
            usage_key = (user_id, tier, i)
            usage_data = self.usage.get(usage_key, {"count": 0, "start_time": now})

            # Reset window if the time has elapsed
            if (now - usage_data["start_time"]) >= window_size:
                usage_data["count"] = 0
                usage_data["start_time"] = now

            # If adding tokens_needed exceeds the limit, queue or reject
            if usage_data["count"] + tokens_needed > limit:
                # Queue the request
                # <Issue>: Queue counter incremented before assignment, causing incorrect position numbering
           
                self.queue_counter += 1
                queue_pos = self.queue_counter
                queued_info = {
                    "status": "queued",
                    "queue_position": queue_pos,
                    "estimated_wait_time": 1,  # simplistic wait estimate
                    "priority_level": priority
                }
                return (False, queued_info)

        # If we can proceed, update usage across all windows
        for i, window_size in enumerate(self.time_windows):
            usage_key = (user_id, tier, i)
            usage_data = self.usage.get(usage_key, {"count": 0, "start_time": now})
            if (now - usage_data["start_time"]) >= window_size:
                usage_data["count"] = 0
                usage_data["start_time"] = now
            usage_data["count"] += tokens_needed
            self.usage[usage_key] = usage_data

        # Build metadata about consumed tokens and remaining for each window
        remaining_tokens = []
        for i, limit in enumerate(token_limits):
            usage_key = (user_id, tier, i)
            usage_data = self.usage[usage_key]
            used = usage_data["count"]
            remain = limit - used
            remaining_tokens.append(remain)

        rate_limit_info = {
            "tokens_consumed": tokens_needed,
            "remaining_tokens": remaining_tokens
        }
        return (True, rate_limit_info)

    def _check_and_consume_requests(self, user_id: str, tier: str):
        """
        Internal helper: checks request usage against the user's tier request limits.
        Returns (proceed: bool, data: dict).
        """
        now = time.time()
        tier_conf = self.tier_configs.get(tier)
        if not tier_conf or "limits" not in tier_conf:
            return (False, {"status": "error", "message": f"No request limits for tier '{tier}'"})

        request_limits = tier_conf["limits"]  # e.g. [2, 10, 100]
        priority = tier_conf.get("priority", 1)

        # Each request consumes 1 usage
        for i, window_size in enumerate(self.time_windows):
            limit = request_limits[i]
            usage_key = (user_id, tier, i)
            usage_data = self.usage.get(usage_key, {"count": 0, "start_time": now})

            # Reset window if the time has elapsed
            if (now - usage_data["start_time"]) >= window_size:
                usage_data["count"] = 0
                usage_data["start_time"] = now

            # Check if adding 1 usage would exceed the limit
            if usage_data["count"] + 1 > limit:
                self.queue_counter += 1
                queue_pos = self.queue_counter
                queued_info = {
                    "status": "queued",
                    "queue_position": queue_pos,
                    "estimated_wait_time": 1,  # simplistic wait estimate
                    "priority_level": priority
                }
                return (False, queued_info)

        # If we can proceed, update usage
        for i, window_size in enumerate(self.time_windows):
            usage_key = (user_id, tier, i)
            usage_data = self.usage.get(usage_key, {"count": 0, "start_time": now})
            if (now - usage_data["start_time"]) >= window_size:
                usage_data["count"] = 0
                usage_data["start_time"] = now
            usage_data["count"] += 1
            self.usage[usage_key] = usage_data

        # Build rate limit metadata
        remaining_requests = []
        for i, limit in enumerate(request_limits):
            usage_key = (user_id, tier, i)
            usage_data = self.usage[usage_key]
            used = usage_data["count"]
            remain = limit - used
            remaining_requests.append(remain)

        rate_limit_info = {
            "requests_consumed": 1,
            "remaining_requests": remaining_requests
        }
        return (True, rate_limit_info)

    def create_limiter(self):
        """
        Factory method that returns a decorator enforcing either token-based or request-based
        rate limiting, depending on the initialized limiter_type.
        """
        if self.limiter_type == "token":
            def token_decorator(func):
                def wrapper(user_id, tier, operation, content, *args, **kwargs):
                    can_proceed, info = self._check_and_consume_tokens(user_id, tier, operation, content)
                    if can_proceed:
                        # Proceed with the original function call
                        result = func(user_id, tier, operation, content, *args, **kwargs)
                        return {
                            "result": result,
                            "rate_limit_info": info
                        }
                    else:
                        # Return the queue or error info
                        return info
                return wrapper
            return token_decorator

        elif self.limiter_type == "request":
            def request_decorator(func):
                def wrapper(user_id, tier, operation, content, *args, **kwargs):
                    can_proceed, info = self._check_and_consume_requests(user_id, tier)
                    if can_proceed:
                        result = func(user_id, tier, operation, content, *args, **kwargs)
                        return {
                            "result": result,
                            "rate_limit_info": info
                        }
                    else:
                        return info
                return wrapper
            return request_decorator

        else:
            raise ValueError("Invalid limiter type. Must be 'token' or 'request'.")


###############################################################################
# Below is an optional helper to demonstrate how one might process inputs and
# produce outputs in the format described in the problem statement.
###############################################################################

def simulate_api_calls(methods, parameters):
    """
    Demonstrates how to handle the Input/Output format described in the prompt.

    methods: list of method names, e.g. ["AIRateLimiter", "create_limiter", "generate_text", ...]
    parameters: list of parameter lists, one for each method call.
    
    Returns a list of results parallel to the method calls:
      - Constructor or void methods return None to match "null" in the output
      - Decorator creation returns a placeholder string (e.g., "decorator_function")
      - Actual API calls return either success or queue info as specified
    """
    results = []
    instance = None
    decorator = None

    # We'll create a dummy function to illustrate usage:
    def generate_text(user_id, tier, operation, content, *args, **kwargs):
        # Just echo the content as if we "generated" it
        return content

    for method, args in zip(methods, parameters):
        if method == "AIRateLimiter":
            # Instantiate the rate limiter
            # args[0] = tier_configs, args[1] = time_windows, args[2] = operation_costs, args[3] = limiter_type
            tier_configs, time_windows, operation_costs, limiter_type = args
            instance = AIRateLimiter(tier_configs, time_windows, operation_costs, limiter_type)
            results.append(None)  # constructor returns null
        elif method == "create_limiter":
            # Must have an instance
            if not instance:
                results.append({"error": "No AIRateLimiter instance exists"})
                continue
            decorator = instance.create_limiter()
            # Return a placeholder to mimic the example (decorator functions can't easily be serialized)
            results.append("decorator_function")
        elif method == "get_token_count":
            # Simple pass-through
            if not instance:
                results.append({"error": "No AIRateLimiter instance exists"})
                continue
            content = args[0]
            count = instance.get_token_count(content)
            results.append(count)
        else:
            # Any other method is presumably an API call like "generate_text"
            if not decorator:
                results.append({"error": f"No decorator created before calling {method}"})
                continue
            # Wrap the dummy function with the chosen decorator
            wrapped_func = decorator(generate_text)
            # Call the wrapped function with the provided arguments
            outputs = wrapped_func(*args)
            results.append(outputs)

    return results
```